# Fsutil suspicious invocation
Detects suspicious parameters of fsutil (deleting USN journal, configuring it with small size..). Might be used by ransomwares during the attack (seen by NotPetya and others)

## Rule Content
```
- title: Fsutil suspicious invocation
  id: add64136-62e5-48ea-807e-88638d02df1e
  description: Detects suspicious parameters of fsutil (deleting USN journal, configuring
    it with small size..). Might be used by ransomwares during the attack (seen by
    NotPetya and others)
  author: Ecco
  date: 2019/09/26
  level: high
  references:
  - https://docs.microsoft.com/en-us/windows-server/administration/windows-commands/fsutil-usn
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    binary_1:
      Image: '*\fsutil.exe'
    binary_2:
      OriginalFileName: fsutil.exe
    selection:
      CommandLine:
      - '* deletejournal *'
      - '* createjournal *'
    condition: (1 of binary_*) and selection
  falsepositives:
  - Admin activity
  - Scripts and administrative tools used in the monitored environment

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\fsutil.exe OR file_name_original:"fsutil.exe") AND process_command_line.keyword:(*\ deletejournal\ * OR *\ createjournal\ *))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()